In [4]:
import tkinter as tk

In [7]:
import os
os.system("Xvfb :1 -screen 0 720x720x16 &")
os.environ['DISPLAY'] = ":2.0"

In [10]:
import PySimpleGUI as sg

layout = [[sg.Text('Filename')],
      [sg.Input(), sg.FileBrowse()],
      [sg.OK(), sg.Cancel()] ]

button, (number,) = sg.FlexForm('Get filename example').LayoutAndRead(layout)

TclError: couldn't connect to display ":2.0"